# Laboratorium 3

In [1]:
from bitarray import bitarray
from copy import deepcopy
from sortedcontainers import SortedList

### Input

In [2]:
names = ["small", "medium", "big", "large", "number"]
inputs=[]
for name in names:
    f = open(name+".txt", "r")
    inputs.append(f.read())
    f.close()

In [3]:
class Node:
    def __init__(self, char='', val=0, par=None):
        self.char=char
        self.val=val
        self.kids=[]
        self.parent=par
        
    def update(self, add):
        self.val+=add
        
    def par(self, other):
        self.parent=other
        
    def kid(self, other):
        self.kids.append(other)
        
    def ch(self, char):
        self.char=char
        
    def __gt__(self, other):
        return self.val<other.val
    
    def __str__(self, indent=0):
        return "%sval=%s%s\n%s" % (" "*indent, self.val, " char="+self.char if len(self.kids)==0 else "",
                                  ''.join(kid.__str__(indent+1) for kid in self.kids))

# Static Huffman

Format:\
Liczba unikalnych znaków - 1 bajt.\
Słownik występownia znaków (1 - na znak, 1 - na liczbę bitów potrzebną do przekazania długości i odpowiednia liczba na liczbę wystąpień, uzupełniona do pełnych bajtów).\
Liczba zer potrzebnych do uzupełnienia ostatniego bajta - 1 bajt.\
Całość tekstu skompresowana

In [4]:
def countchars(txt):
    count = dict()
    for char in txt:
        if char not in count:
            count[char]=1
        else:
            count[char]+=1
    return {k: v for k, v in sorted(count.items(), key=lambda x: -x[1])}

In [5]:
def stahuff(txt):
    count = countchars(txt)
    trees = SortedList()
    for char in count:
        trees.add(Node(char, count[char]))
    while len(trees) > 1:
        tmp = [trees.pop(), trees.pop()]
        tree = Node(None, tmp[0].val+tmp[1].val)
        tree.kid(tmp[0])
        tree.kid(tmp[1])
#         print(tmp[0].val, tmp[1].val, tree.val)
        tmp[0].par(tree)
        tmp[1].par(tree)
        trees.add(tree)
    return trees[0]

Funkcja stahuff - tworzy drzewo dla algorytmu statycznego huffmana

In [6]:
def makedict(tree, res=dict(), cur=bitarray()):
    if len(tree.kids)==0:
        res[tree.char]=cur
    else:
        for ix, kid in enumerate(tree.kids):
            tmp=deepcopy(cur)
            tmp.append(ix)
            makedict(kid, res, tmp)
    return res

Funkcja makedict - tworzy słownik, przydatny przy kompresji

In [7]:
tree=stahuff(inputs[0])
# tree=stahuff("abracadabra")
# print(tree)
# print(countchars(inputs[0]))
prp=makedict(tree, dict(), bitarray())
# prp

In [8]:
def descr(prp):
    res = bitarray()
    res += "{0:b}".format(len(prp))
    res = bitarray('0'*(8-len(res)))+res
    for char, code in prp.items():
        a=bitarray()
        a.frombytes(char.encode())
        a = bitarray('0'*(8-len(a)))+a
#         print(a, a.tobytes())
        res+=a
        tmp = "{0:b}".format(len(code))
        tmp = '0'*(8-len(tmp))+tmp
#         print(tmp)
        res += bitarray(tmp)
        res += code
#     print(res)
    return res

Funkcja descr - tworzy nagłówek pliku o foramcie.st - słownik

In [9]:
def compsta(txt=None, name="small", outname="cmp", prp=None):
    if txt==None:
        with open(name+".txt", 'r') as file:
            txt=file.read()
    if prp==None:
        prp=makedict(stahuff(txt), dict(), bitarray())
    desc=descr(prp)#, "{0:b}".format((len(txt))))
    res=bitarray()
    for char in txt:
        res+=prp[char]
    le = len(res)+len(desc)
    res += '0'*(8-le+int(le/8)*8)
    tmp = "{0:b}".format(8-le+int(le/8)*8)
    tmp = '0'*(8-len(tmp)+int(len(tmp)/8)*8)+tmp
    res = desc+bitarray(tmp)+res
#     print(desc, tmp, res)
#     print(res)
    
    with open(outname+".st", 'wb') as file:
        res.tofile(file)
    return res

Funkcja compsta - kompresja pozwalająca na różne wersje(nie trzeba z pliku czytać), może odczytać plik, skompresować i zapisać do nowego pliku oraz zwraca wynik jako string

In [10]:
# comp = compsta(txt=inputs[0])
comp = compsta(name="small", outname="smallcmp")
comp = compsta(name="medium", outname="mediumcmp")
comp = compsta(name="big", outname="bigcmp")
comp = compsta(name="large", outname="largecmp")
comp = compsta(name="number", outname="numbercmp")
# comp

In [11]:
def maketree(txt):
    uniqs = txt[0:8]
    uni = 0
    for i in uniqs:
        uni = (uni << 1) | i
    ix = 8
    T = dict()
    for _ in range(uni):
        le = txt[ix+8:ix+8+8]
        length = 0
        for i in le:
            length = (length << 1) | i
        char = 0
        for i in txt[ix:ix+8]:
            char = (char << 1) | i
        T[chr(char)]=txt[ix+8+8:ix+8+8+length]
        ix+=8+8+length
    head = Node()
    for char, code in T.items():
        cur = head
        for way in code:
            if len(cur.kids) == 0:
                tmp=Node()
                cur.kid(tmp)
                tmp.par(cur)
                tmp=Node()
                cur.kid(tmp)
                tmp.par(cur)
            cur = cur.kids[1 if way else 0]
        cur.ch(char)
    return head, ix

Funkcja maketree - tworzy drzewo z ze słownika zawartego w skompresowanym pliku i zwraca index do końca słownika w tekście

In [12]:
def read(txt, tree):
    res=""
    cur = tree
    for char in txt:
        cur = cur.kids[1 if char else 0]
        if len(cur.kids)==0:
            res+=cur.char
            cur=tree
    return res

Funkcja read - tłumaczy skompresowany text przy pomocy drzewa, dekompresuje tekst

In [13]:
def decompsta(name="cmp", outname="decomp", txt=None):
    if txt==None:
        txt = bitarray()
        with open(name+".st", 'rb') as file:
            txt.fromfile(file)
    tree, ix = maketree(txt)
    off = txt[ix:ix+8]
    ov=0
    for bit in off:
        ov = (ov<<1) | bit # ov<8
    res = read(txt[ix+8:-ov], tree)
#     print(res)
    with open(outname+".txt", 'w') as file:
        file.write(res)
    return res
#     print(tree)

Funkcja decompsta - dekompresuje, skompresowany text - można go dostarczyć plikiem lub stringiem i zapisuje wynik do pliku oraz go zwraca

In [14]:
# decomp(txt=comp)
# decompsta()
decomp = decompsta(name="smallcmp", outname="smalldecmp")
decomp = decompsta(name="mediumcmp", outname="mediumdecmp")
decomp = decompsta(name="bigcmp", outname="bigdecmp")
decomp = decompsta(name="largecmp", outname="largedecmp")
decomp = decompsta(name="numbercmp", outname="numberdecmp")

In [15]:
def test(txt='abracadabra'):
    prp=makedict(stahuff(txt), dict(), bitarray())
    res=bitarray()
    desc=descr(prp)
    for char in txt:
        res+=prp[char]
    le = len(res)+len(desc)
    res += '0'*(8-le+int(le/8)*8)
    tmp = "{0:b}".format(8-le+int(le/8)*8)
    tmp = '0'*(8-len(tmp)+int(len(tmp)/8)*8)+tmp
    comp = desc+bitarray(tmp)+res
    print(comp)
    tree, ix = maketree(comp)

    ov=0
    for bit in comp[ix:ix+8]:
        ov = (ov<<1) | bit # ov<8
    res = read(comp[ix+8:-ov], tree)
    print(res)

Funkcja test - sprawdza czy kompresja i dekompresja działa dla podanego tekstu

In [16]:
test()

bitarray('0000010101100001000000010011000100000001010011001000000010011000110001100000100110101110010000000111110000001101011101101011000101110000')
abracadabra


Aby skompresować plik należy wywołać polecenie(funkcja również zwraca skompresowany kod bitowy):\
compsta(name="small", outname="smallcmp")

Aby plik skompresowany rozpakować, należy wywołać polecenie(funkcja zwraca rozpakowany tekst):\
decompsta(name="cmp", outname="decomp")

W obu przypadkach można zmienić name i outname, aby zmienić rozszerzenia plików należy poszperać w tych funkcjach(nie wydaje mi się to konieczne - możnaby wgl usunąć +'.txt')

In [17]:
comp = compsta(name="small", outname="smallcmp")
decomp = decompsta(name="smallcmp", outname="decomp")

Kompresja i dekompresja działają, kompresja redukuje rozmiar pliku o mniej więcej połowę.

# Dynamic/Adaptive Huffman

Format:\
Adaptacyjny kod huffmana\
Dopełnienie zerami (<8 bitów)\
Liczba bitów dopełnionych (1 bajt - trochę się zmarnuje bo starczyłyby 3)

In [18]:
class Tree:
    def __init__(self):
        self.zero = Node('00', val=0)
        self.root = self.zero
        self.nodes = []
        self.seen = [None]*(2**8) # should be enough for our characters

    def code(self, char, node, code=''):
        if len(node.kids)==0:
            return code if node.char == char else ''
        else:
            tmp = self.code(char, node.kids[0], code+'0')
            if not tmp:
                tmp = self.code(char, node.kids[1], code+'1')
            return tmp
    
    def find(self, val):
        for node in reversed(self.nodes):
            if node.val == val:
                return node
            
    def swap(self, n1, n2):
        i1, i2 = self.nodes.index(n1), self.nodes.index(n2)
        self.nodes[i1], self.nodes[i2] = self.nodes[i2], self.nodes[i1]
#         tmp = n1.parent
#         n1.parent = n2.parent
#         n2.parent = tmp
        n1.parent, n2.parent = n2.parent, n1.parent
        
        if n1.parent.kids[0]==n2:
            n1.parent.kids[0]=n1
        else:
            n1.parent.kids[1]=n1
        if n2.parent.kids[0]==n1:
            n2.parent.kids[0]=n2
        else:
            n2.parent.kids[1]=n2
        
#         for i in range(2):
#             if n1.parent.kids[i] == n2:
#                 n1.parent.kids[i] = n1
#             if n2.parent.kids[i] == n1:
#                 n2.parent.kids[i] = n2
                
    def add(self, char):
        node = self.seen[ord(char)]
        
        if node is None:
            upd = Node('', val=1, par=self.zero.parent)
            new = Node(char=char, val=1, par=upd)
            upd.kid(self.zero)
            upd.kid(new)
            self.zero.par(upd)
            
            if upd.parent is not None:
                upd.parent.kids[0]=upd
            else:
                self.root=upd
                
            self.nodes.insert(0, upd)
            self.nodes.insert(0, new)
            self.seen[ord(char)]=new
            node = upd.parent
        
        while node is not None:
            toswap = self.find(node.val)
            
            if node is not toswap and node is not toswap.parent and node.parent is not toswap:
#                 print(char)
#                 print('pre:\n',self.root)
                self.swap(node, toswap)
#                 print('post:\n',self.root)
                
            node.val += 1
            node=node.parent
            
    def comp(self, txt):
        res = ''
        for char in txt:
#             print('\n\tstart',char, end=":")
            if self.seen[ord(char)]:
                res += self.code(char, self.root)
            else:
                res += self.code('00', self.root)
#                 print(self.code('00', self.root))
                res += bin(ord(char))[2:].zfill(8)
#             print(res)
#             print(self.root)
#             print(char, ':', res)
            self.add(char)
        length = len(res)
        res += '0'*(8-length+int(length/8)*8)
        res += bin(8-length+int(length/8)*8)[2:].zfill(8)
        return bitarray(res)
    
    def decomp(self, txt):
        off=0
        for bit in txt[-8:]:
            off = (off << 1) | bit
        txt = txt[:-8-off]
        
        res = ''
        char = 0
        for bit in txt[:8]:
            char = (char << 1) | bit
        char = chr(char)
        self.add(char)
        res+=char
        node = self.root
        
        ix=8
        while ix < len(txt):
            node = node.kids[1 if txt[ix] else 0]
            char = node.char
            
            if char:
                if char == '00':
                    char = 0
                    for bit in txt[ix+1:ix+1+8]:
                        char = (char << 1) | bit 
                    char = chr(char)
                    ix+=8
                res += char
                self.add(char)
                node = self.root
            ix+=1
        return res

Struktura tree, implementująca różne metody pomocne i konieczne przy algorytmie dynamicznego huffmana

In [19]:
comp=Tree().comp('abracadabra')
print(comp)
print(Tree().decomp(comp))

bitarray('011000010011000100001110010010001100011011000110010001101100000000000100')
abracadabra


In [20]:
def compad(name='small', outname='wowcomp'):
    with open(name+".txt", 'r') as file:
        txt=file.read()
    with open(outname+".ad", 'wb') as file:
        Tree().comp(txt).tofile(file)
    return Tree().comp(txt)

Funkcja compad - dostaje 2 pliki, otwiera i czyta pierwszy, zapisuje skompresowaną wartość do drugiego

In [21]:
def decompad(name='wowcomp', outname='wowdecomp'):
    txt=bitarray()
    with open(name+'.ad', 'rb') as file:
        txt.fromfile(file)
    with open(outname+'.txt', 'w') as file:
        file.write(Tree().decomp(txt))
    return Tree().decomp(txt)

Funkcja decompad - dostaje 2 pliki, otwiera i czyta pierwszy, zapisuje zdekompresowaną wartość do drugiego

In [22]:
with open('small.txt', 'r') as file:
    txt=file.read()

print(len(txt))
len(Tree().comp(txt))

1027


4736

In [23]:
comp = compad()
decomp = decompad()

In [24]:
comp = compad(name="small", outname="smallwowcmp")
comp = compad(name="medium", outname="mediumwowcmp")
comp = compad(name="big", outname="bigwowcmp")
comp = compad(name="large", outname="largewowcmp")
comp = compad(name="number", outname="numberwowcmp")

In [25]:
decomp = decompad(name="smallwowcmp", outname="smallwowdecmp")
decomp = decompad(name="mediumwowcmp", outname="mediumwowdecmp")
decomp = decompad(name="bigwowcmp", outname="bigwowdecmp")
decomp = decompad(name="largewowcmp", outname="largewowdecmp")
decomp = decompad(name="numberwowcmp", outname="numberwowdecmp")

Aby skompresować plik należy wywołać polecenie(funkcja również zwraca skompresowany kod bitowy):\
compad(name="small", outname="wowcomp")

Aby plik skompresowany rozpakować, należy wywołać polecenie(funkcja zwraca rozpakowany tekst):\
decompad(name="wowcomp", outname="wowdecomp")

W obu przypadkach można zmienić name i outname, aby zmienić rozszerzenia plików należy poszperać w tych funkcjach(nie wydaje mi się to konieczne - możnaby wgl usunąć +'.txt')

In [26]:
comp = compad(name="small", outname="wowcomp")
decomp = decompad(name="wowcomp", outname="wowdecomp")

# Comparison

In [27]:
from os.path import getsize as size
from filecmp import cmp as diff

Oba sposoby działają, bezstrarnie kompresują dane i pozwalają je rozpakować.

In [28]:
assert size("small.txt")==size("smalldecmp.txt")==size("smallwowdecmp.txt")
assert size("medium.txt")==size("mediumdecmp.txt")==size("mediumwowdecmp.txt")
assert size("big.txt")==size("bigdecmp.txt")==size("bigwowdecmp.txt")
assert size("large.txt")==size("largedecmp.txt")==size("largewowdecmp.txt")
assert size("number.txt")==size("numberdecmp.txt")==size("numberwowdecmp.txt")
assert diff("smalldecmp.txt", "smallwowdecmp.txt") and diff("small.txt", "smallwowdecmp.txt")
assert diff("mediumdecmp.txt", "mediumwowdecmp.txt") and diff("medium.txt", "mediumwowdecmp.txt")
assert diff("bigdecmp.txt", "bigwowdecmp.txt") and diff("big.txt", "bigwowdecmp.txt")
assert diff("largedecmp.txt", "largewowdecmp.txt") and diff("large.txt", "largewowdecmp.txt")
assert diff("numberdecmp.txt", "numberwowdecmp.txt") and diff("number.txt", "numberwowdecmp.txt")

Porównajmy rozmiary plików po skompresowaniu:\
Podane procentowo - współczynnik kompresji - skompresowane/nieskompresowane

In [29]:
def comppow(file="small"):
    f=file+'.txt'
    sta=file+'cmp.st'
    ada=file+'wowcmp.ad'
    print(f"for file {f} with size {size(f)} bytes compression powers are")
    print(f"Static: {(1-size(sta)/size(f))*100}%\nAdaptive: {(1-size(ada)/size(f))*100}%\n")

In [30]:
comppow("small")
comppow("medium")
comppow("big")
comppow("large")
comppow("number")

for file small.txt with size 1027 bytes compression powers are
Static: 36.02726387536514%
Adaptive: 42.35637779941578%

for file medium.txt with size 10273 bytes compression powers are
Static: 45.42003309646646%
Adaptive: 46.11116519030468%

for file big.txt with size 100329 bytes compression powers are
Static: 46.435228099552475%
Adaptive: 46.499018230023225%

for file large.txt with size 1048576 bytes compression powers are
Static: 45.30763626098633%
Adaptive: 45.2946662902832%

for file number.txt with size 1048576 bytes compression powers are
Static: 56.214332580566406%
Adaptive: 56.10466003417969%



Ogólnie oba sposoby kompresują pliki bardzo podobnie, jeden jest czasem trochę lepszy od drugiego, jednak zależy to od danych kompresowanych. W naszym przypadku możemy zauważyć że największe pliki były lepiej skompresowane przez statycznego huffmana, natomiast przy wszystkich pozostałych adaptacyjny kompresował lepiej.

Zmierzmy czasy kompresji i dekompresji (wraz z zapisem i odczytem z pliku):

In [31]:
def tim(file='small'):
    f=file+'.txt'
    print(f"For file {f} with size {size(f)} bytes compression and decompresion times for static:")
    print("\nCompresion:")
    %timeit index = compsta(name=file, outname="comptmpsta")
    print("\nDecompresion:")
    %timeit index = decompsta(name="comptmpsta", outname="decomptmpsta")
    
    print(f"\n\nFor file {f} with size {size(f)} compression and decompresion times for adaptive:")
    print("\nCompresion:")
    %timeit index = compad(name=file, outname="comptmpada")
    print("\nDecompresion:")
    %timeit index = decompad(name="comptmpada", outname="decomptmpada")
# %timeit index = 

In [32]:
tim('small')

For file small.txt with size 1027 bytes compression and decompresion times for static:

Compresion:
1.31 ms ± 8.35 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

Decompresion:
1.28 ms ± 5.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


For file small.txt with size 1027 compression and decompresion times for adaptive:

Compresion:
50.8 ms ± 1.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Decompresion:
29.1 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
tim('medium')

For file medium.txt with size 10273 bytes compression and decompresion times for static:

Compresion:
4.75 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Decompresion:
9.46 ms ± 69.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


For file medium.txt with size 10273 compression and decompresion times for adaptive:

Compresion:
536 ms ± 4.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Decompresion:
198 ms ± 611 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
tim('big')

For file big.txt with size 100329 bytes compression and decompresion times for static:

Compresion:
36.9 ms ± 465 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Decompresion:
89.5 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


For file big.txt with size 100329 compression and decompresion times for adaptive:

Compresion:
5.48 s ± 15.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Decompresion:
1.72 s ± 11.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
tim('large')

For file large.txt with size 1048576 bytes compression and decompresion times for static:

Compresion:
374 ms ± 6.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Decompresion:
944 ms ± 4.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


For file large.txt with size 1048576 compression and decompresion times for adaptive:

Compresion:
58 s ± 592 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Decompresion:
19.3 s ± 365 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
tim('number')

For file number.txt with size 1048576 bytes compression and decompresion times for static:

Compresion:
348 ms ± 2.05 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Decompresion:
782 ms ± 4.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


For file number.txt with size 1048576 compression and decompresion times for adaptive:

Compresion:
21.4 s ± 2.11 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

Decompresion:
12.6 s ± 59.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Jak widać metoda statyczna, pozwala szybciej kompresować i dekompresować pliki, natomiast wymaga podania założeń na początku (częstość występowania/prawdopodobieństwo wystąpienia litery) co wymaga dodatkowego przejścia przez cały plik bądź dodatkowego założenia.\
Medota dynamiczna pozwala kompresować i dekompresować plik "w biegu", nie wymaga dodatkowych założeń, jednak trwa trochę dłużej (można ją przyspieszyć ulepszająć metodę add i find)